In [ ]:
from read_penguin import load_3D_data, cell_center
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import geometry as geo
from Interpolate3DSpherical import triInterpolate, interpolate3DSpherical, binSearch

In [ ]:
def density(x, y, z, data):
    xVals = cell_center(data[1])
    yVals = cell_center(data[2])
    zVals = cell_center(data[3])
    dens = data[4]
    r = [x, y, z]
    den = interpolate3DSpherical(xVals, yVals, zVals, dens, r)
    return den

In [ ]:
def finddens3D(x, y, z, data, planetPosition):
    
    #transforming input planet coordinates to cartesian
    sphepolar = (x,y,z)
    cartesian = geo.sphericalToCartesian(sphepolar, dim=3)
    
    #transforming to star-centric spherical and interpolating
    StarCentricCart = np.array(cartesian) + np.array(geo.sphericalToCartesian(planetPosition, dim = 3))
    starCentric = geo.cartesianToSpherical(StarCentricCart, dim = 3)
    den = density(starCentric[0], starCentric[1], starCentric[2], data)
    return den


In [ ]:
#Loading density data

xres = 384
yres = 768
zres = 216
data = load_3D_data("/scratch/mtalukd/sc/scratch/3DAdiabaticParameterChange/Gamma1.4/", xres, yres, zres, "h50_1p1J_e0_PPM4", 0)


In [ ]:
#creating a 2D grid around midplane
planetCoord = (1, np.pi, np.pi*0.5)
planetCoordCart = geo.sphericalToCartesian(planetCoord, dim = 3)


def populateX(i, data):
    return 0 + 0.1*i/500

def populateY(i, data):
    return 2*np.pi*i/500


planetX = np.ndarray(500)
planetY = np.ndarray(500)


for i in range(500):
    planetX[i] = populateX(i, data)
    planetY[i] = populateY(i, data)
    

DensNew = np.ndarray((500,500))

for i in range(500):
    for j in range(500):
        d = finddens3D(planetX[i], planetY[j], np.pi*0.5, data, planetCoord)
        DensNew[i,j] = d
print(DensNew)
#polar = np.pi*0.5 - 0.005
#d = finddens3D(0.00001, 1, polar, data, planetCoord)


In [ ]:
plt.figure()
plt.pcolor(planetY, planetX, DensNew, cmap = "RdBu")
plt.title("Density")
plt.ylabel("R")
plt.xlabel("Theta Azi (Rad)")
plt.colorbar()
plt.savefig("gamma1.4radiusvazi.png", dpi=300)
plt.show()

In [ ]:
fig, ax = plt.subplots(subplot_kw = {'projection': 'polar'})
ax.grid(False)
plt.pcolormesh(data[2], data[1], np.transpose(data[4][-1]), cmap = "RdBu")
plt.colorbar()
plt.savefig("raw_gamma1.4planet24.png", dpi=300)
plt.show()

In [ ]:
#plotting vertical circles 

planetCoord = (1, np.pi, np.pi*0.5)
planetCoordCart = geo.sphericalToCartesian(planetCoord, dim = 3)
imax = 100
kmax = 100

def populateX(i, imax, data):
    return 0.0 + 0.1*i/imax

def populateZ(i, kmax, data):
    return np.pi*i/kmax


planetX = np.ndarray(imax)
planetZ = np.ndarray(kmax)


for i in range(imax):
    planetX[i] = populateX(i, imax, data)
    planetZ[i] = populateZ(i, kmax, data)
    

DensNew1 = np.ndarray((imax,kmax))

for i in range(imax):
    for k in range(kmax):
        d = finddens3D(planetX[i], np.pi, planetZ[k] , data, planetCoord)
        DensNew1[i,k] = d
        #print(d)
print(DensNew1)

In [ ]:
plt.figure()
plt.pcolor(planetZ, planetX, DensNew1, cmap = "RdBu")
plt.title("Density")
plt.ylabel("R")
plt.xlabel("Theta Polar (Rad)")
plt.clim(0,2)
plt.colorbar()
plt.savefig("gamma1.4radiusvpolar.png", dpi=300)

In [ ]:
print(finddens3D(0.002, np.pi, 1.6 , data, planetCoord))